In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

In [4]:
print("=== BƯỚC 1: KIỂM TRA DỮ LIỆU ===")
df = pd.read_csv("USA_Housing.csv")
print("Kích thước:", df.shape)
print("\nInfo:")
df.info()
print("\nThiếu dữ liệu:")
print(df.isna().sum())
print("\nMô tả nhanh:")
print(df.describe())

=== BƯỚC 1: KIỂM TRA DỮ LIỆU ===
Kích thước: (5000, 7)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
 6   Address                       5000 non-null   object 
dtypes: float64(6), object(1)
memory usage: 273.6+ KB

Thiếu dữ liệu:
Avg. Area Income                0
Avg. Area House Age             0
Avg. Area Number of Rooms       0
Avg. Area Number of Bedrooms    0
Area Population                 0
Price                           0
Address                    

In [5]:
print("\n=== BƯỚC 2: TẠO ĐẶC TRƯNG MỚI ===")
req = ["Avg. Area Income", "Avg. Area House Age", 
       "Avg. Area Number of Rooms", "Avg. Area Number of Bedrooms", 
       "Area Population", "Price"]

for c in req:  
    assert c in df.columns, f"Thiếu cột: {c}"

# Tạo đặc trưng room_price_ratio
df["room_price_ratio"] = (
    df["Avg. Area Number of Rooms"] / 
    df["Avg. Area Number of Bedrooms"].replace({0: np.nan})
)
print("Đã tạo đặc trưng mới: room_price_ratio")


=== BƯỚC 2: TẠO ĐẶC TRƯNG MỚI ===
Đã tạo đặc trưng mới: room_price_ratio


In [6]:
print("\n=== BƯỚC 3: TÁCH DỮ LIỆU TRAIN/TEST ===")
target = "Price"

# Kiểm tra cột phân loại
categorical_candidates = [c for c in ["Address"] if c in df.columns]
print(f"Cột phân loại: {categorical_candidates}")

feature_cols = [
    "Avg. Area Income", "Avg. Area House Age",
    "Avg. Area Number of Rooms", "Avg. Area Number of Bedrooms",
    "Area Population", "room_price_ratio"
] + categorical_candidates

X = df[feature_cols].copy()
y = df[target].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"Kích thước X_train: {X_train.shape}, X_test: {X_test.shape}")


=== BƯỚC 3: TÁCH DỮ LIỆU TRAIN/TEST ===
Cột phân loại: ['Address']
Kích thước X_train: (4000, 7), X_test: (1000, 7)


In [7]:
print("\n=== BƯỚC 4: ĐỊNH NGHĨA PIPELINE ===")
numeric_features = [c for c in X.columns if c not in categorical_candidates]
categorical_features = categorical_candidates

print(f"Cột số: {numeric_features}")
print(f"Cột phân loại: {categorical_features}")

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),   
    ("scaler", MinMaxScaler())                      
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_features),
        ("cat", cat_pipe, categorical_features)
    ],
    remainder="drop"
)


=== BƯỚC 4: ĐỊNH NGHĨA PIPELINE ===
Cột số: ['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', 'Area Population', 'room_price_ratio']
Cột phân loại: ['Address']


In [8]:

selector = SelectKBest(score_func=f_regression, k=3)
linreg = LinearRegression()

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("select", selector),
    ("model", linreg)
])


In [9]:
print("\n=== BƯỚC 6: HUẤN LUYỆN & ĐÁNH GIÁ ===")
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R² (test): {r2:.4f}")
print(f"MAE (test): ${mae:,.2f}")


=== BƯỚC 6: HUẤN LUYỆN & ĐÁNH GIÁ ===
R² (test): 0.7950
MAE (test): $126,906.14


In [10]:
# Bước 7 — Truy vết đặc trưng được chọn và hệ số
print("\n=== BƯỚC 7: PHÂN TÍCH MÔ HÌNH ===")
# Lấy tên đặc trưng sau preprocess
preprocessor = pipeline.named_steps["preprocess"]
preprocessor.fit(X_train)  # Đảm bảo preprocessor đã được fit

try:
    feature_names = preprocessor.get_feature_names_out()
except Exception as e:
    print(f"Không lấy được feature names: {e}")
    # Fallback: tạo tên đặc trưng
    n_features_after_preprocess = pipeline.named_steps["select"].scores_.shape[0]
    feature_names = np.array([f"feature_{i}" for i in range(n_features_after_preprocess)])

# Chỉ số cột được SelectKBest giữ lại
selected_idx = pipeline.named_steps["select"].get_support(indices=True)
selected_names = feature_names[selected_idx]

print("\nTop-3 đặc trưng được chọn:")
for i, name in enumerate(selected_names, 1):
    score = pipeline.named_steps["select"].scores_[selected_idx[i-1]]
    print(f"{i}. {name} (score: {score:.2f})")

# Trích hệ số của LinearRegression
coef = pipeline.named_steps["model"].coef_
intercept = pipeline.named_steps["model"].intercept_

print("\nHệ số LinearRegression:")
for name, weight in zip(selected_names, coef):
    print(f"- {name}: {weight:,.2f}")
print(f"Intercept: {intercept:,.2f}")

# Đánh giá thêm trên tập train để kiểm tra overfitting
y_train_pred = pipeline.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

print(f"\nĐánh giá trên tập train:")
print(f"R² (train): {r2_train:.4f}")
print(f"MAE (train): ${mae_train:,.2f}")

# Phân tích importance của các đặc trưng gốc
print("\n=== PHÂN TÍCH ẢNH HƯỞNG CỦA CÁC ĐẶC TRƯNG GỐC ===")
print("Scores từ SelectKBest (trước khi chọn):")
original_features = numeric_features + categorical_features
for i, feature in enumerate(original_features):
    if i < len(pipeline.named_steps["select"].scores_):
        print(f"- {feature}: {pipeline.named_steps['select'].scores_[i]:.2f}")


=== BƯỚC 7: PHÂN TÍCH MÔ HÌNH ===

Top-3 đặc trưng được chọn:
1. num__Avg. Area Income (score: 2802.19)
2. num__Avg. Area House Age (score: 1010.36)
3. num__Area Population (score: 792.61)

Hệ số LinearRegression:
- num__Avg. Area Income: 1,874,064.88
- num__Avg. Area House Age: 1,114,344.80
- num__Area Population: 1,053,998.98
Intercept: -947,300.34

Đánh giá trên tập train:
R² (train): 0.7987
MAE (train): $126,576.13

=== PHÂN TÍCH ẢNH HƯỞNG CỦA CÁC ĐẶC TRƯNG GỐC ===
Scores từ SelectKBest (trước khi chọn):
- Avg. Area Income: 2802.19
- Avg. Area House Age: 1010.36
- Avg. Area Number of Rooms: 491.00
- Avg. Area Number of Bedrooms: 128.79
- Area Population: 792.61
- room_price_ratio: 2.52
- Address: 0.01
